In [24]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


In [25]:

columName=['userID','age','gender','occupation','code']

In [26]:
df_users=pd.read_csv('C:/Users/sansi/Desktop/dat/u.user',encoding='Latin',sep='|',names=columName)
df_movies=pd.read_csv('C:/Users/sansi/Desktop/dat/movies.dat',encoding='Latin',sep='::',names=['movie_id', 'title', 'movie_genre'])
df_ratings=pd.read_csv('C:/Users/sansi/Desktop/dat/ratings.dat',encoding='Latin',sep='::',names=['userID','movie_id','rate','tims'])
df_tags=pd.read_csv('C:/Users/sansi/Desktop/dat/tags.dat',encoding='Latin',sep='::',names=['userID','movie_id','tag','tims'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='pyt

In [27]:
df_movies

,movie_id,title,movie_genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [28]:
df_movies=df_movies.dropna(how='any')
df_users=df_users.dropna(how='any')
df_ratings=df_ratings.dropna(how='any')
df_tags=df_tags.dropna(how='any')
df_movies_test=df_movies.copy()
df_movies_gender=df_movies.copy()
from sklearn.model_selection import train_test_split

In [29]:
df_users['age'] = pd.cut(df_users['age'], bins=[18,30, 45,55,65, np.inf], labels=[1,2,3,4,5], right=False)#categorie age

In [30]:
df_ratings['rate'].replace([1, 2,3,4,5],[-1, -1, 1, 1, 1],inplace=True)

In [31]:
df=df_movies_test.merge(df_ratings,on='movie_id')
df=df_users.merge(df,on='userID')
df.set_index('movie_id')
rate_trainset,rate_testset =train_test_split(df, test_size=0.2, random_state=0)

df_movies_gender=df_movies.copy()
rate_trainset=rate_trainset.dropna(how='any')
rate_trainset

,userID,age,gender,occupation,code,movie_id,title,movie_genre,rate,tims
122508,791,2,M,educator,20064,969,"African Queen, The (1951)",Action|Adventure|Romance|War,1,980621362
31301,211,5,M,salesman,32605,3627,Carnival of Souls (1962),Horror|Thriller,1,976907804
27924,195,2,M,scientist,93555,849,Escape from L.A. (1996),Action|Adventure|Sci-Fi|Thriller,-1,1028259212
55978,371,2,M,engineer,99206,3481,High Fidelity (2000),Comedy,1,976314895
139105,890,2,M,student,97301,344,Ace Ventura: Pet Detective (1994),Comedy,1,975470240
...,...,...,...,...,...,...,...,...,...,...
122579,791,2,M,educator,20064,1569,My Best Friend's Wedding (1997),Comedy|Romance,1,982854640
41993,284,2,M,executive,92629,3564,"Flintstones in Viva Rock Vegas, The (2000)",Children's|Comedy,-1,976576736
97639,655,3,F,healthcare,60657,35,Carrington (1995),Drama|Romance,-1,975699752
95939,644,3,M,retired,63645,1912,Out of Sight (1998),Action|Crime|Romance,1,975720463


In [32]:
#df=df_movies_test.merge(df_ratings[df_ratings['userID']==6],on='movie_id')
#df=df_users.merge(df[df['userID']==6],on='userID')
#df.set_index('movie_id')

In [33]:
df_movies=pd.DataFrame(df_movies.loc[:,['movie_id','movie_genre']])
df_movies = pd.concat([df_movies, df_movies.movie_genre.str.get_dummies(sep='|')], axis=1)
df_movies = df_movies.drop(['movie_genre'],axis=1)
df_movies=pd.DataFrame(df_movies.set_index('movie_id'))

In [34]:
n=df_movies.sum(axis=1) 
df_movies_cat=df_movies.divide(n.iloc[:],axis=0)
#len(nonLIST) pour obtenire la taille de la liste 
df_movies_cat

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.333333,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3950,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_movies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3950,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [36]:
#ponderation tfidf
n=df_movies.sum(axis=0) 
df_df=pd.Series(n,index=df_movies.columns)
def idf(i):
    k=np.log(df_movies.size/i)
    return k  
df_idf=pd.Series(idf(df_df[0:18]))
df_movies_penderation=df_idf[:]*df_movies
df_movies_penderation.head(10)

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,6.500775,5.629282,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.000000,5.509288,0.000000,5.629282,0.000000,0.000000,0.0,0.000000,6.935227,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,4.999877,0.0,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,3.775103,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
6,4.934145,0.000000,0.000000,0.000000,0.000000,5.802877,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,4.956256,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,4.064658,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,4.999877,0.0,0.000000,0.0,0.0
8,0.000000,5.509288,0.000000,5.629282,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
9,4.934145,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [37]:
df_ratings=df_ratings.drop('tims',axis=1)
df_users=df_users.drop('code',axis=1)
#build the user profile
#inferer le centre d'interet d'un utilisateur 

In [38]:
df_ratings_user=rate_trainset[rate_trainset['userID']==1]
df_poids=df_movies_cat[df_movies_cat.index.isin(df_ratings_user.movie_id)]

   

In [39]:
#df_poids.values[0][0]*df['rate'].values[0]

In [40]:

df_user_rate=df_poids.T.mul(df['rate']).sum(axis=1)


In [41]:
user_preference=df_user_rate.T


In [42]:
user_preference=pd.DataFrame(user_preference)

In [43]:
user_preference

,0
Action,0.916667
Adventure,0.833333
Animation,5.166667
Children's,5.500000
Comedy,4.083333
Crime,0.666667
Documentary,0.000000
Drama,8.166667
Fantasy,1.000000
Film-Noir,0.000000


In [44]:
c=cosine_similarity(user_preference.T,df_movies_penderation)
R = np.argsort(c)[:, ::-1]
R
df['rate']

0         1
1         1
2         1
3         1
4         1
         ..
146455    1
146456    1
146457    1
146458    1
146459    1
Name: rate, Length: 146460, dtype: int64

In [45]:
def user_table(i) :
    #df=df_movies_test.merge(df_ratings[df_ratings['userID']==i],on='movie_id')
    #df=df_users.merge(df[df['userID']==i],on='userID')
    #df.set_index('movie_id')
    df=rate_trainset[rate_trainset['userID']==i]
    return df
user_table(1)

,userID,age,gender,occupation,code,movie_id,title,movie_genre,rate,tims
6,1,1,M,technician,85711,588,Aladdin (1992),Animation|Children's|Comedy|Musical,1,978824268
0,1,1,M,technician,85711,1,Toy Story (1995),Animation|Children's|Comedy,1,978824268
34,1,1,M,technician,85711,1962,Driving Miss Daisy (1989),Drama,1,978301753
32,1,1,M,technician,85711,1907,Mulan (1998),Animation|Children's,1,978824330
15,1,1,M,technician,85711,919,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,1,978301368
3,1,1,M,technician,85711,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,978300760
25,1,1,M,technician,85711,1246,Dead Poets Society (1989),Drama,1,978302091
50,1,1,M,technician,85711,3114,Toy Story 2 (1999),Animation|Children's|Comedy,1,978302174
26,1,1,M,technician,85711,1270,Back to the Future (1985),Comedy|Sci-Fi,1,978300055
8,1,1,M,technician,85711,595,Beauty and the Beast (1991),Animation|Children's|Musical,1,978824268


In [46]:
def homme():
    df_homme=rate_trainset[rate_trainset['gender']=='M']
    df_homme_rate=df_movies_cat.T.mul(df_homme['rate']).sum(axis=1)
    df_homme_rate=pd.DataFrame(df_homme_rate)
    homme=cosine_similarity(df_homme_rate.T,df_movies_penderation)
    h=np.argsort(homme)[:, ::-1]
    recommendations_homme = [i for i in h[0] if i not in df_homme['movie_id'].values]
    preference_homme=df_movies_gender['title'][recommendations_homme]
    return preference_homme

In [47]:
def femme():
    df_femme=rate_trainset[rate_trainset['gender']=='F']
    df_femme_rate=df_movies_cat.T.mul(df_femme['rate']).sum(axis=1)
    df_femme_rate=pd.DataFrame(df_femme_rate)
    femme=cosine_similarity(df_femme_rate.T,df_movies_penderation)
    f=np.argsort(femme)[:, ::-1]
    recommendations_femme = [i for i in f[0] if i not in df_femme['movie_id'].values]
    preference_femme=df_movies_gender['title'][recommendations_femme]
    return preference_femme

In [48]:
def age(i):
    if i<=5:
        df_age=rate_trainset[rate_trainset['age']==i]
        df_age_rate=df_movies_cat.T.mul(df_age['rate']).sum(axis=1)
        df_age_rate=pd.DataFrame(df_age_rate)
        age=cosine_similarity(df_age_rate.T,df_movies_penderation)
        g=np.argsort(age)[:, ::-1]
        recommendations_age = [i for i in g[0] if i not in df_age['movie_id'].values]
        preference_age=df_movies_gender['title'][recommendations_age]
    return preference_age

In [49]:
def recommander(userID):
    df_ratings_user=rate_trainset[rate_trainset['userID']==userID]
    df_poids=df_movies_cat[df_movies_cat.index.isin(df_ratings_user.movie_id)]
    df_user_rate=df_poids.T.mul(user_table(userID).rate).sum(axis=1)
    user_preference=df_user_rate.T
    user_preference=pd.DataFrame(user_preference)
    c=cosine_similarity(user_preference.T,df_movies_penderation)
    R = np.argsort(c)[:, ::-1]
    interet = [i for i in R[0] if i not in user_table(userID)['movie_id'].values]
    centre_interet=df_movies_test['title'][interet]
    df_users_id=df_users[df_users['userID']==userID]
    j=df_users_id['age'].values
    if df_users_id['gender'].values=='M':
        homme_age=age(j[0])[age(j[0]).isin(homme())]
        recommandation=centre_interet[centre_interet.isin(homme_age)]
    else:
        femme_age=age(j[0])[age(j[0]).isin(femme())]
        recommandation=centre_interet[centre_interet.isin(femme_age)]
     
    return recommandation 

In [50]:
recommander(3)

1324                            Carrie (1976)
1319             Bride of Frankenstein (1935)
1318             Bram Stoker's Dracula (1992)
1316                        Body Parts (1991)
1315                       Blood Beach (1981)
                        ...                  
2592          It Came from Outer Space (1953)
2591     Thing From Another World, The (1951)
2588    Rocky Horror Picture Show, The (1975)
2584                    Son of Dracula (1943)
0                            Toy Story (1995)
Name: title, Length: 674, dtype: object

In [51]:
def metric(y_test,y_pred):
    from sklearn import metrics
    precision=metrics.precision_score(y_test,y_pred)
    rappel=metrics.recall_score(y_test,y_pred)
    list_metrics=[precision,rappel]
    return list_metrics

In [52]:
def rappel():
    k=[]    
    for l in df_users['userID'].head(20).values:
        testset=rate_testset[rate_testset['userID']==l]
        pertinant=testset['title'][testset['title'].isin(recommander(l))]
        testset=testset[testset['rate']==1]
        rappel=pertinant.size/len(testset)
    #rappel=nombre du doc pertinant retourner(pertinant) /nombre totale de doc pertinant
    #precision=nombre de doc pertinant delectionner/nombre totale de doc selectionner
        k.append(rappel)
    return k


In [23]:
sum(k)/20


NameError: name 'k' is not defined

In [58]:
def precision():
    p=[]    
    for l in rate_trainset['userID'].head(20).values:
        testset=rate_testset[rate_testset['userID']==l]
        tp=testset['title'][testset['title'].isin(recommander(l))]
        testset=testset[testset['rate']==1]
        fp=100
        precision=tp.size/20
    #rappel=nombre du doc pertinant retourner(pertinant) /nombre totale de doc pertinant
    #precision=nombre de doc pertinant delectionner/nombre totale de doc selectionner
        p.append(precision)
    return p 

In [ ]:
precision()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_users['userID'].head(30),precision())
plt.plot(df_users['userID'].head(30),rappel())
plt.xlabel('users')
plt.ylabel('precision')
plt.title('figure1:precision')
plt.show()